<a href="https://colab.research.google.com/github/Chandan0731/bioinformatics_lab/blob/main/Experiment_2_sequence_allignment_fasta%26blast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FASTA**

In [1]:
!pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.1 MB/s eta 0:00:00


In [2]:
from Bio import Entrez, SeqIO

# Always tell NCBI who you are
Entrez.email = "chandanksshetty@gmail.com"

# Replace these with your real gene IDs
id1 = "PV539470.1"
id2 = "PV539469.1"

# Fetch sequences from NCBI
handle1 = Entrez.efetch(db="nucleotide", id=id1, rettype="fasta", retmode="text")
handle2 = Entrez.efetch(db="nucleotide", id=id2, rettype="fasta", retmode="text")

with open("gene1.fasta", "w") as f:
    f.write(handle1.read())

with open("gene2.fasta", "w") as f:
    f.write(handle2.read())


In [3]:
def read_fasta(file_path):
    """Reads a FASTA file and returns a dictionary of sequences."""
    sequences = {}
    current_sequence_name = None
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                current_sequence_name = line[1:]
                sequences[current_sequence_name] = ''
            elif current_sequence_name:
                sequences[current_sequence_name] += line
    return sequences


def simple_sequence_comparison(seq1, seq2, match_score=1, mismatch_penalty=-1, gap_penalty=-2):
    """Performs a simple character-by-character comparison of two sequences."""
    score = 0
    alignment_seq1 = ""
    alignment_seq2 = ""
    i, j = 0, 0

    while i < len(seq1) and j < len(seq2):
        if seq1[i] == seq2[j]:
            score += match_score
            alignment_seq1 += seq1[i]
            alignment_seq2 += seq2[j]
        else:
            score += mismatch_penalty
            alignment_seq1 += seq1[i]
            alignment_seq2 += seq2[j]
        i += 1
        j += 1

    while i < len(seq1):
        score += gap_penalty
        alignment_seq1 += seq1[i]
        alignment_seq2 += '-'
        i += 1
    while j < len(seq2):
        score += gap_penalty
        alignment_seq1 += '-'
        alignment_seq2 += seq2[j]
        j += 1

    return score, alignment_seq1, alignment_seq2


# === 🧬 Use your real FASTA files here ===
sequences1 = read_fasta('gene1.fasta')
sequences2 = read_fasta('gene2.fasta')

# Assuming each file has one sequence
seq1_name = list(sequences1.keys())[0]
seq2_name = list(sequences2.keys())[0]
sequence1 = sequences1[seq1_name]
sequence2 = sequences2[seq2_name]

score, aligned_seq1, aligned_seq2 = simple_sequence_comparison(sequence1, sequence2)

print(f"Sequence 1 ({seq1_name}): {sequence1}")
print(f"Sequence 2 ({seq2_name}): {sequence2}")
print("\nSimple Comparison Results:")
print(f"Alignment Score: {score}")
print(f"Aligned Sequence 1: {aligned_seq1}")
print(f"Aligned Sequence 2: {aligned_seq2}")


Sequence 1 (PV539470.1 Plasmopara ampelopsidis voucher BPI 911272 cytochrome oxidase subunit II (cox2) gene, partial cds; mitochondrial): GCAAAATGGGGTTTTCAAGATCCAGCAACTCCAGTTATGGAAGGTATTATTAACTTTCATCATGATATAATGTTTTTTTTAATTATGATTACTGTATTCGTTTGTTGGATGTTATTTAGAGTTGTTACTCTTTTTGATGAAAAAAAAAACAAAATACCGGCAACTGTTGTACACGGAGCAACCATTGAAATTATTTGGACTTCAATTCCCGCTTTGATTTTACTAATTATTGCTATACCATCGTTTGCATTGCTGTATTCTATGGATGAAATAATCGATCCTATTATTACCTTAAAAGTAATTGGAAGTCAGTGGTATTGGAGTTATGAATACTCCGATAATTTAGAGTTTTCAGACGAACCTTTAATTTTTGATAGTTATATGGTACCGGAAAATGATTTAGCAATAGGTCAATTTAGATTATTAGAAGTTGATAATCGTGTAATAGTTCCAACGAATAGTCATATCCGTGTATTGATTACTGCATCAGATGTTTTACATTCATGGGCAATACCGTCATTGGGGATAAAATTAGATGCCTGTCCAGGCCGT
Sequence 2 (PV539469.1 Plasmopara ampelopsidis voucher BPI 911272 cytochrome oxidase subunit I (cox1) gene, partial cds; mitochondrial): AAATCATAAAGATATCGGAACTTTATATTTAATTTTTAGTGCTTTTGCTGGTGTTGTTGGTACCACCTTATCCCTTTTAATACGAATGGAATTAGCACAGCCTGGTAATCAAATTTTTATGGGAAATCATCAATTATACAATGTTGTTGTTACTGCACATGCTTTTGTAATG

**BLAST**

In [4]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# Define the two sequence IDs you want to BLAST
sequence_ids = ["PV539470.1", "PV539470.1"] # Replace with your desired sequence IDs

# Perform BLAST for each sequence
for seq_id in sequence_ids:
    print(f"Performing BLAST for sequence ID: {seq_id}")
    # Perform the BLAST search
    result_handle = NCBIWWW.qblast(
        "blastn",  # BLAST program (nucleotide blast)
        "nt",      # Database (nucleotide database)
        seq_id     # Query sequence ID
    )

    # Parse the BLAST results
    blast_records = NCBIXML.parse(result_handle)

    # Print some information about the hits
    for blast_record in blast_records:
        print(f"Query: {blast_record.query}")
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                print(f"  Alignment: {alignment.title}")
                print(f"  Length: {alignment.length}")
                print(f"  E-value: {hsp.expect}")
                print(f"  Query start: {hsp.query_start}, Query end: {hsp.query_end}")
                print(f"  Subject start: {hsp.sbjct_start}, Subject end: {hsp.sbjct_end}")
                print(f"  Score: {hsp.score}")
                print(f"  Identities: {hsp.identities}")
                print(f"  Gaps: {hsp.gaps}")
                print(f"  Query: {hsp.query[0:75]}...")
                print(f"  Match: {hsp.match[0:75]}...")
                print(f"  Sbjct: {hsp.sbjct[0:75]}...")
                print("-" * 80)

    result_handle.close()

Performing BLAST for sequence ID: PV539470.1
Query: Plasmopara ampelopsidis voucher BPI 911272 cytochrome oxidase subunit II (cox2) gene, partial cds; mitochondrial
  Alignment: gi|3008897273|gb|PV539470.1| Plasmopara ampelopsidis voucher BPI 911272 cytochrome oxidase subunit II (cox2) gene, partial cds; mitochondrial
  Length: 552
  E-value: 0.0
  Query start: 1, Query end: 552
  Subject start: 1, Subject end: 552
  Score: 1104.0
  Identities: 552
  Gaps: 0
  Query: GCAAAATGGGGTTTTCAAGATCCAGCAACTCCAGTTATGGAAGGTATTATTAACTTTCATCATGATATAATGTTT...
  Match: |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
  Sbjct: GCAAAATGGGGTTTTCAAGATCCAGCAACTCCAGTTATGGAAGGTATTATTAACTTTCATCATGATATAATGTTT...
--------------------------------------------------------------------------------
  Alignment: gi|2293318243|gb|OK631957.1| Plasmopara sp. isolate H6 cytochrome oxidase subunit 2 (cox2) gene, partial cds; mitochondrial >gi|2293318251|gb|OK631961.1| Plasmopara sp. isolate U3